In [10]:
%run tire_detection.ipynb

In [3]:
from __future__ import print_function
import sys
import cv2
from random import randint
 
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
 
def createTrackerByName(trackerType):
  # Create a tracker based on tracker name
    if trackerType == trackerTypes[0]:
        tracker = cv2.TrackerBoosting_create()
    elif trackerType == trackerTypes[1]: 
        tracker = cv2.TrackerMIL_create()
    elif trackerType == trackerTypes[2]:
        tracker = cv2.TrackerKCF_create()
    elif trackerType == trackerTypes[3]:
        tracker = cv2.TrackerTLD_create()
    elif trackerType == trackerTypes[4]:
        tracker = cv2.TrackerMedianFlow_create()
    elif trackerType == trackerTypes[5]:
        tracker = cv2.TrackerGOTURN_create()
    elif trackerType == trackerTypes[6]:
        tracker = cv2.TrackerMOSSE_create()
    elif trackerType == trackerTypes[7]:
        tracker = cv2.TrackerCSRT_create()
    else:
        tracker = None
        print('Incorrect tracker name')
        print('Available trackers are:')
        for t in trackerTypCSRTes:
            print(t)
     
    return tracker

In [4]:
def intersected(b1, b2):
    # If one rectangle is on left side of other 
    if((b1[0] > b2[0] + b2[2]) or (b2[0] > b1[0] + b1[2])): 
        return False
  
    # If one rectangle is above other 
    if((b1[1] > b2[1] + b2[3]) or (b2[1] > b1[1] + b1[3])): 
        return False
    
    return True


def is_not_included(bbox, bboxes):
    for b in bboxes:
        if intersected(bbox, b):
            return False
    return True

In [5]:
def multiple_object_tracking(tracker_name, video_path):
    

    cap = cv2.VideoCapture(video_path)

    success, frame = cap.read()
    if not success:
        print('Failed to read video')
        sys.exit(1)

    bboxes = []
    colors = [] 


    trackerType = tracker_name   

    multiTracker = cv2.MultiTracker_create()
    boxes = []

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        circles = detect_tires(frame)
        if circles is not None:
            for (x, y, r) in circles:
                box_top = ((max(0, x-r), max(0, y-r)))
                bbox = (box_top[0], box_top[1], min(frame.shape[1] - box_top[0], 2*r), min(frame.shape[0] - box_top[1], 2*r))
                if is_not_included(bbox, boxes):
                    multiTracker.add(createTrackerByName(trackerType), frame, bbox)


        success, boxes = multiTracker.update(frame)

        for i, newbox in enumerate(boxes):
            p1 = (int(newbox[0]), int(newbox[1]))
            p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
            cv2.rectangle(frame, p1, p2, (0, 0, 0), 2, 1)

        cv2.imshow('MultiTracker', frame)


      # quit on ESC button
        if cv2.waitKey(1) & 0xFF == 27:  # Esc pressed
            break
    cv2.destroyAllWindows()

In [12]:
import sys
import cv2

def single_object_tracking(tracker_name, video_path):
    tracker = createTrackerByName(tracker_name)
    video = cv2.VideoCapture(video_path)
 
    if not video.isOpened():
        print("Could not open video")
        sys.exit()
        

    frame = None
    bbox = None
    ind = 0
    
    
    while True:
        # Read first frame
        ok, frame = video.read()
        
        if not ok:
            print('Cannot read video file')
            sys.exit()
            
        res = detect_tires(frame)
        
        if res is None: 
            continue
            
        (x, y, r) = res[0]
        box_top = ((max(0, x-r), max(0, y-r)))
        bbox = (box_top[0], box_top[1], min(frame.shape[1] - box_top[0], 2*r), min(frame.shape[0] - box_top[1], 2*r))
        break

 
    ok = tracker.init(frame, bbox)
 
    while True:
        ok, frame = video.read()
        
        if not ok:
            break
          
        timer = cv2.getTickCount()
 
        ok, bbox = tracker.update(frame)
 
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
 
        if ok:
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        else :
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
 
        cv2.putText(frame, tracker_name + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);
     
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);
 
        cv2.imshow("Tracking", frame)
 
        if cv2.waitKey(1) & 0xff == 27:
            break
    cv2.destroyAllWindows()

In [15]:
multiple_object_tracking("CSRT", "video_folder/ProcessedODSample2.mp4")

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
